In [1]:
import boto3
import json

In [2]:
def read_json(file='./iamcred.json'):
    response = dict()
    response['status'] = True
    try:
        with open(file, 'r') as file:
            response['data'] = json.load(file)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [3]:
def get_dynamodb_resource(data):
    response = dict()
    response['status'] = True
    try:
        response['data'] = boto3.resource(
            service_name = data['service_name'],
            region_name = data['region_name'],
            aws_access_key_id = data['aws_access_key_id'],
            aws_secret_access_key = data['aws_secret_access_key']
        )
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [4]:
def create_table(dynamodb, data):
    response = dict()
    response['status'] = True
    try:
        dynamodb.create_table(
            TableName = data['TableName'],
            KeySchema = data['KeySchema'],
            AttributeDefinitions = data['AttributeDefinitions'],
            ProvisionedThroughput = data['ProvisionedThroughput']
        )
        response['data'] = 'successful!'
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [5]:
def main(iamcred_file, table_data):
    response = dict()
    response['status'] = True
    try:
        resp = read_json(iamcred_file)
        if resp['status'] == False:
            raise Exception(resp['data'])
        creds = resp['data']
        
        resp = get_dynamodb_resource(creds)
        if resp['status'] == False:
            raise Exception(resp['data'])
        dynamodb = resp['data']
        
        resp = create_table(dynamodb, table_data)
        if resp['status'] == False:
            raise Exception(resp['data'])
        response['data'] = resp['data']
        
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [6]:
iamcred_file = './iamcred.json'
TableName = 'Electricity'
table_data = {
    'TableName': TableName,
    'KeySchema': [
        {
            'AttributeName': 'turned_on', 
            'KeyType': 'HASH'
        }
    ],
    'AttributeDefinitions': [
        {
           'AttributeName': 'turned_on',
           'AttributeType': 'N'
        },
    ],
    'ProvisionedThroughput': {
        'ReadCapacityUnits':5,
        'WriteCapacityUnits':5
    }
}
main(iamcred_file, table_data)

{'status': True, 'data': 'successful!'}